Denne del installere manim og de dele der er nødvendige for at kunne benytte manim-biblioteket.

Tager ca. 4 min at kører. Man kører en celle ved at taste enten **ctr + enter** eller ved at klikke på den lille playknap i øverste venstre hjørne

In [1]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://ppa.launchpad.net/c2d4u.team/

Når ovenstående kode er kørt skal kørselstiden genstartes dette gøres i menuen *Runtime* og vælge *Genstart kørselstid*.

Herefter skal vi indlæse manim-biblioteket og dette gøres med følgende kode.

In [1]:
from manim import *

Manim Community v0.17.2

Nu er alt klart til at lave animationer i manim. Koden her under generere et plot at sandsynligheden for at få krone for hvert kast man gør. Det er lavet med en random funktion så hvert output bliver forskelligt. Jeg har lavet en graf der opdateres løbende for hvert kast, hvor man selv kan sætte hvor mange kast der skal laves.

Vær obs på at dette ikke er den mest optimale måde at kode dette på hvorfor tiden også er lang. For 1000 kast vil det nok tage i underkanten af 2 timer ved middel kvalitet. Hvis man gerne vil have høj billedkvalitet vil tiden være betydelig højere.

In [2]:
%%manim -qm -v WARNING test2

import random

class test2(Scene):
  def construct(self):
    #LATEX STUFF
    #myTemplate = TexTemplate()
    config["tex_template"].add_to_preamble(r"\usepackage[utf8]{inputenc}",prepend=True)
    config["tex_template"].add_to_preamble(r"\usepackage[scaled=0.95]{helvet}")
    config["tex_template"].add_to_preamble(r"\renewcommand\familydefault{\sfdefault}")
    config["tex_template"].add_to_preamble(r"\usepackage[T1]{fontenc}")
    config["tex_template"].add_to_preamble(r"\usepackage[helvet]{sfmath}")
    config["tex_template"].add_to_preamble(r"\usepackage{amsmath,bm}")
    config["tex_template"].add_to_preamble(r"\usepackage{upgreek}")

    #COLORS
    MYBLUE = rgb_to_color(hex_to_rgb("#0066cc"))
    HIGHLIGHT_SHADE = rgb_to_color(hex_to_rgb("#cce6ff"))
    HIGHLIGHT_LINE = rgb_to_color(hex_to_rgb("#66b3ff"))
    INDICATE_COLOR = rgb_to_color(hex_to_rgb("#f8cd82"))
    BACKGROUND = rgb_to_color(hex_to_rgb("#141B2D"))
    BLUE1 = rgb_to_color(hex_to_rgb("#6BE0EC"))
    LIGHT_PINK = rgb_to_color(hex_to_rgb("#F4E1F5"))
    PINK3 = rgb_to_color(hex_to_rgb("#F4E1F5"))
    PINK2 = rgb_to_color(hex_to_rgb("#FACAFD"))
    PINK1 = rgb_to_color(hex_to_rgb("#E79BEC"))
    CORRECT_COLOR = rgb_to_color(hex_to_rgb("#09D542"))
    MISTAKE_COLOR = rgb_to_color(hex_to_rgb("#FF5454"))
    MYGRAY = rgb_to_color(hex_to_rgb("#F3F4F5"))
    MYGRAY_OUT = rgb_to_color(hex_to_rgb("#878EA0"))
    CINNABAR = rgb_to_color(hex_to_rgb("#EF3E36"))
    APRICOT = rgb_to_color(hex_to_rgb("#f7664d"))#("#faac91"))#("#FAC9B8"))
    AERO = rgb_to_color(hex_to_rgb("#77B6EA"))
    LIGHT_GREEN = rgb_to_color(hex_to_rgb("#81F495"))
    MING = rgb_to_color(hex_to_rgb("#1e3738"))#("#294b4d"))#("#38686A"))

    #BACKGROUND
    self.camera.background_color=MING

    #Variabler
    n = 100 #Denne variabel kan ændre for at får flere eller færre kast med mønten
    heads = 0
    tails = 0

    prob = []
    flips = []

    plots=[]
    graphs = []
    lines = []
    ns =[]
    P_all = []

    #Tæller til at vise antal kast
    decimal = DecimalNumber(
            1,
            show_ellipsis=False,
            num_decimal_places=0,
            include_sign=False,
            color = AERO
    ).to_corner(UP+RIGHT)
    
    #Tekst på skærmen
    antal_kast = Tex(r"\# kast: ")#.next_to(decimal,LEFT)

    titel = Tex(r"Sandsynlighed ", "for ", r"krone ", r"ved ", r"kast ", r"med ", r"en ", r"\bf mønt").to_edge(UP)
    titel[2].set_color(GOLD)
    
    #Generere sandsynlighederne og gemmer dem i variablen flips
    for i in range(n):
      coin = random.randint(0,1)
      if coin == 0:
        heads += 1
      else:
        tails += 1
      prob.append(heads/(i+1))
      flips.append(i+1)

    #Laver plot af sandsynlighederne. Er indelt i tre intervaller for at ændre på ticks op x-aksen
    for i in range(n):
      turn = i+1
      if 0 <= turn <= 20: 
        ax = Axes(
            x_range=[0,1+i],
            y_range=[0,1.2,0.5],
            y_length=4,
            x_axis_config={"numbers_to_include": np.arange(0, 1+i, 1)},
            y_axis_config={"numbers_to_include": np.arange(0, 1.2, 0.5)}
          )
        plots.append(ax)
        labels = ax.get_axis_labels(x_label=Tex(r"\footnotesize n"), y_label=Tex(r"\footnotesize P(X=1)"))
        x_val =[]
        y_val =[]
        for flip in range(turn):
          x_val.append(flips[flip])
          y_val.append(prob[flip])
      #print(x_val,y_val) #-- used for debugging
        graph = ax.plot_line_graph(x_values=x_val,y_values=y_val, vertex_dot_radius=0.08/(1.00696**i),line_color = GOLD,vertex_dot_style=dict(fill_color = GOLD))
        graphs.append(graph)
        line = ax.plot(lambda x: 0.5, color=APRICOT, x_range = [0,turn])
        lines.append(DashedVMobject(line))
        ns.append(turn)
      if 20 < turn <= 200: 
        ax = Axes(
            x_range=[0,1+i,10],
            y_range=[0,1.2,0.5],
            y_length=4,
            x_axis_config={"numbers_to_include": np.arange(0, 1+i, 10)},
            y_axis_config={"numbers_to_include": np.arange(0, 1.2, 0.5)}
          )
        plots.append(ax)
        labels = ax.get_axis_labels(x_label=Tex(r"\footnotesize n"), y_label=Tex(r"\footnotesize P(X=1)"))
        x_val =[]
        y_val =[]
        for flip in range(turn):
          x_val.append(flips[flip])
          y_val.append(prob[flip])
      #print(x_val,y_val) #-- used for debugging
        graph = ax.plot_line_graph(x_values=x_val,y_values=y_val, vertex_dot_radius=0.08/(1.00696**i),line_color = GOLD,vertex_dot_style=dict(fill_color = GOLD))
        graphs.append(graph)
        line = ax.plot(lambda x: 0.5, color=APRICOT, x_range = [0,turn])
        lines.append(DashedVMobject(line))
        ns.append(turn)
      if 200 < turn <= 1000:
        ax = Axes(
            x_range=[0,1+i,100],
            y_range=[0,1.2,0.5],
            y_length=4,
            x_axis_config={"numbers_to_include": np.arange(0, 1+i, 100)},
            y_axis_config={"numbers_to_include": np.arange(0, 1.2, 0.5)}
          )
        plots.append(ax)
        labels = ax.get_axis_labels(x_label=Tex(r"\footnotesize n"), y_label=Tex(r"\footnotesize P(X=1)"))
        x_val =[]
        y_val =[]
        for flip in range(turn):
          x_val.append(flips[flip])
          y_val.append(prob[flip])
      #print(x_val,y_val) #-- used for debugging
        graph = ax.plot_line_graph(x_values=x_val,y_values=y_val, vertex_dot_radius=0.08/(1.00696**i),line_color = GOLD,vertex_dot_style=dict(fill_color = GOLD))
        graphs.append(graph)
        line = ax.plot(lambda x: 0.5, color=APRICOT, x_range = [0,turn])
        lines.append(DashedVMobject(line))
        ns.append(turn)
      
      #print("kast:",i) #-- used for debugging
      #print(len(plots)) #-- used for debugging
      #print(len(graphs)) #-- used for debugging
      #print(len(lines)) #-- used for debugging

    #plot initial condition
    decimal.align_to(plots[0],RIGHT).shift(DOWN)
    antal_kast.next_to(decimal,LEFT)
    self.add(plots[0],graphs[0],lines[0],decimal,labels,antal_kast,titel)
    self.wait(0.1)

    

    #update contions
    for i in range(n-1):
      #print("start") #-- used for debugging
      plots[0].become(plots[1+i])
      #print("plot") #-- used for debugging
      graphs[0].become(graphs[1+i])
      #print("graph") #-- used for debugging
      lines[0].become(lines[1+i])
      #print("lines") #-- used for debugging
      decimal.set_value(ns[1+i])
      #print("plot",i) #-- used for debugging
      self.wait(0.05)

    self.wait(3) # Venter 3 sekunder inden animationen går videre. Man kan ændre denne hvis man ønsker længere eller kortere tid
 
    #Laver 6 nye datasæt for sandsynligheden
    for j in range(6):
    #  print("start") #-- used for debugging
      heads = 0
      tails = 0

      Ps = []
      for i in range(n):
        coin = random.randint(0,1)
        if coin == 0:
          heads += 1
        else:
          tails += 1
        Ps.append(heads/(i+1))
      P_all.append(Ps)

      #print("datasæt:",j) #-- used for debugging

    #Plotter de 6 nye datasæt
    dataset = []
    for data in range(len(P_all)):
    #  print("start2")
      data_point = plots[n-1].plot_line_graph(
          x_values=ns,
          y_values=P_all[data],
          line_color=PURPLE_C,
          add_vertex_dots=False,
          #vertex_dot_radius=0.08,
          vertex_dot_style=dict(
              stroke_width=3,
              fill_color=PURPLE,
              stroke_opacity = 0.2,
              fill_opacity = 0.2
            ),
          stroke_width = 4,
          stroke_opacity = 0.2,
        ).set_z_index(-2)
    #  print("start2.1") #-- used for debugging
      dataset.append(data_point)
    
    #print(len(dataset)) #-- used for debugging
    
    #Tilføjer de 6 nye datasæt til grafen
    for graph in range(len(dataset)):
      self.add(dataset[graph])

    
    self.wait(5) #Venter 5 sekunder inden animationen slutter